In [1]:
import numpy as np
import sys
import os
#import tensorflow as tf

import io
from gensim.models import word2vec

from io import StringIO
from pprint import pprint

from nltk.tokenize import word_tokenize
import nltk


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import types
from IPython import get_ipython #for import notebook
from nbformat import read #for import notebook
from IPython.core.interactiveshell import InteractiveShell #for import notebook

In [3]:
def find_notebook(fullname, path=None): #for import notebook
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [4]:
class NotebookLoader(object): #for import notebook
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [5]:
sys.meta_path.append(NotebookFinder())

In [6]:
def sent_loader(sentence):
    result = []
    for elem in sentence.split(' '):
        if elem != '':
            result = np.append(result, elem)
    return result,len(result)

In [ ]:
wv_model_en = word2vec.Word2Vec(size=100, window=5, min_count=5, workers=4)
wv_model_en = word2vec.Word2Vec.load('/home/jongsu/Desktop/persona_dialogue/wv_parameter/dialogue_wv')

In [16]:
#tokenize dialogues_test.txt
results = []
with open('/home/jongsu/Desktop/persona_dialogue/ijcnlp_dailydialog/test/dialogues_test.txt') as inputfile:
    for line in inputfile:
        results.append(line.strip().split('__eou__'))
        
full_idx = []
transed = []
i = 0

while(i < len(results)):
    
    dial_to_idx = []
    t_dial_to_idx = []
    j = 0
    while(j < len(results[i])):
    
        a,b = sent_loader(results[i][j])
        idx = []
        t_idx = []
        k = 0
        while(k < b):

            try:
                idx.append(wv_model_en.wv.vocab[a[k]].index) #good
                t_idx.append(1)
            except: #no data in voc
                #print(a[k]) good
                idx.append(a[k])
                t_idx.append(0)
            k = k + 1
            
            
        dial_to_idx.append(idx)
        t_dial_to_idx.append(t_idx)
        j = j + 1
        
    i = i + 1
    full_idx.append(dial_to_idx)
    transed.append(t_dial_to_idx)

#save dialogue
np.save("/home/jongsu/Desktop/persona_dialogue/indexed_dataset/i_dt_test",full_idx,allow_pickle=True)
np.save("/home/jongsu/Desktop/persona_dialogue/indexed_dataset/t_dt_test",transed,allow_pickle=True)

In [8]:
#tokenize dialogues_train.txt
results = []
with open('/home/jongsu/Desktop/persona_dialogue/ijcnlp_dailydialog/train/dialogues_train.txt') as inputfile:
    for line in inputfile:
        results.append(line.strip().split('__eou__'))
        
full_idx = []
transed = []
i = 0

while(i < len(results)):
    
    dial_to_idx = []
    t_dial_to_idx = []
    j = 0
    while(j < len(results[i])):
    
        a,b = sent_loader(results[i][j])
        idx = []
        t_idx = []
        k = 0
        while(k < b):

            try:
                idx.append(wv_model_en.wv.vocab[a[k]].index) #good
                t_idx.append(1)
            except: #no data in voc
                #print(a[k]) good
                idx.append(a[k])
                t_idx.append(0)
            k = k + 1
            
            
        dial_to_idx.append(idx)
        t_dial_to_idx.append(t_idx)
        j = j + 1
        
    i = i + 1
    full_idx.append(dial_to_idx)
    transed.append(t_dial_to_idx)

#save dialogue
np.save("/home/jongsu/Desktop/persona_dialogue/indexed_dataset/i_dt",full_idx,allow_pickle=True)
np.save("/home/jongsu/Desktop/persona_dialogue/indexed_dataset/t_dt",transed,allow_pickle=True)

[['Say , Jim , how about going for a few beers after dinner ? ',
  ' You know that is tempting but is really not good for our fitness . ',
  ' What do you mean ? It will help us to relax . ',
  " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ",
  " I guess you are right.But what shall we do ? I don't feel like sitting at home . ",
  ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ',
  " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ",
  ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ',
  " Good.Let ' s go now . ",
  ' All right . ',
  ''],
 ['Can you do push-ups ? ',
  " Of course I can . It's a piece of cake ! Believe it or not , I can do 30 push-ups a minute . ",
  " Really ? I think that's impossible ! ",
  ' You mean 30 push-ups ? ',
  '

In [17]:
#save action test
results = []
with open('/home/jongsu/Desktop/persona_dialogue/ijcnlp_dailydialog/test/dialogues_act_test.txt') as inputfile:
    for line in inputfile:
        results.append(line.strip())
results[:10]
np.save("/home/jongsu/Desktop/persona_dialogue/indexed_dataset/d_act_test",results,allow_pickle=True)

In [14]:
#save action
results = []
with open('/home/jongsu/Desktop/persona_dialogue/ijcnlp_dailydialog/train/dialogues_act_train.txt') as inputfile:
    for line in inputfile:
        results.append(line.strip())
results[:10]
np.save("/home/jongsu/Desktop/persona_dialogue/indexed_dataset/d_act",results,allow_pickle=True)

In [18]:
#save emotion test
results = []
with open('/home/jongsu/Desktop/persona_dialogue/ijcnlp_dailydialog/test/dialogues_emotion_test.txt') as inputfile:
    for line in inputfile:
        results.append(line.strip())

np.save("/home/jongsu/Desktop/persona_dialogue/indexed_dataset/d_emo_test",results,allow_pickle=True)

In [15]:
#save emotion
results = []
with open('/home/jongsu/Desktop/persona_dialogue/ijcnlp_dailydialog/train/dialogues_emotion_train.txt') as inputfile:
    for line in inputfile:
        results.append(line.strip())

np.save("/home/jongsu/Desktop/persona_dialogue/indexed_dataset/d_emo",results,allow_pickle=True)